Lets read in the Data

In [1]:
import os 
os.getcwd()

'C:\\Users\\Crist\\Rossman_Drugstore_TimeSeries_Analysis\\Rossmann-Store-Sales-Project\\Jupyter_Notebooks'

In [2]:
import os 
import pandas as pd

sales_df = pd.read_csv(r'C:\Users\Crist\Rossman_Drugstore_TimeSeries_Analysis\Rossmann-Store-Sales-Project\data\train.csv')
store_df = pd.read_csv(r'C:\Users\Crist\Rossman_Drugstore_TimeSeries_Analysis\Rossmann-Store-Sales-Project\data\store.csv')
df = pd.merge(store_df,sales_df,on='Store', how='inner')

C:\Users\Crist\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Investigating Data integrity
Checking the data set for structural errors. These may include inconsistencies in attribute units,typos, mislabeled classes etc.

In [3]:
df.isna().sum()

Store                             0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
dtype: int64

In [4]:
df.dtypes

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
dtype: object

df.column.value_counts() comes in great use to check data integrity object data types are susceptible

In [5]:
df.PromoInterval.value_counts()

Jan,Apr,Jul,Oct     293122
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: PromoInterval, dtype: int64

In [6]:
try:
    df.Date = pd.to_datetime(df.Date, format = '%Y-%m-%d')
except TypeError:
    df.Date = pd.to_datetime(df.Date)

no error setting the format equal to %Y/%m/%d meaning its gtg

In [7]:
df.loc[df.Store ==1]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-07-31,5263,555,1,1,0,1
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2015-07-30,5020,546,1,1,0,1
2,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2015-07-29,4782,523,1,1,0,1
3,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2,2015-07-28,5011,560,1,1,0,1
4,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,1,2015-07-27,6102,612,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,6,2013-01-05,4997,635,1,0,0,1
938,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2013-01-04,4486,619,1,0,0,1
939,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2013-01-03,4327,578,1,0,0,1
940,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2013-01-02,5530,668,1,0,0,1


In [8]:
df.StoreType.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [9]:
df.StoreType = df.StoreType.astype('category')
df.Assortment = df.Assortment.astype('category')

After checking each object type data column we found that **StateHoliday was read in and was assigned some mislabeled classes**

In [10]:
df.StateHoliday.unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [11]:
df.StateHoliday.replace('0',0,inplace= True)
df.StateHoliday.replace(0,'no',inplace=True)

There are stores with Competition Distance noted but no CompetitionOpenSinceMonth or CompetitionOpenSinceYear.
For those stores we are going to fill in those two columns respectively with the first - month,year - that competition distance was noted for the store respectively. We are going to create a column to indicate the number of months competions has been opened for an observed date.This will help model the number of months competition has been "opened" for an observed date.

In [12]:
comp_wodate = df.loc[df.CompetitionDistance.notnull() & df.CompetitionOpenSinceYear.isna(),:].copy()

There are 351 such stores that have Competition Distance value but no CompetitionOpenSinceYear/Month

In [13]:
len(comp_wodate.groupby('Store'))

351

In [14]:
comp_wdate = df.loc[(df.CompetitionDistance.notnull())&(df.CompetitionOpenSinceYear.notnull())].copy()

There are 761 stores that have competition distance and competition open since

In [15]:
len(comp_wdate.groupby('Store'))

761

**Does not add to 115 meaning we hav e 1115-(761+351 = 1112) = 3 stores that are different lets check them out**

In [16]:
comp_nodate = []
for name,group in comp_wodate.groupby('Store'):
    comp_nodate.append(name)
comp_withdate = []
for name,group in comp_wdate.groupby('Store'):
    comp_withdate.append(name)
    

There are no interlapping Stores thus **these 351 stores CompetitionOpenSinceMonth & CompetitionOpenSinceYear can be filled with 2013 and 1 respectively**. Which is the earliest date for each of the 351 stores that is recored with having a CompetitionDistance Value but no CompetitionOpenSinceMonth & CompetitionOpenSinceYear.

In [17]:
set(comp_withdate).intersection(set(comp_nodate))

set()

So we find that 3 stores 291, 622, and 879 had no Competition Distance, CompetitionOpenSinceYear, CompetitionOpenSinceMonth

In [18]:
odd_stores = df[(~df.Store.isin(comp_withdate)) & (~df.Store.isin(comp_nodate))].copy()
for name,group in odd_stores.groupby('Store'):
    print(name)

291
622
879


Our purpose is to generalize as much as possible to stores that have Competition thus these stores will be removed.

In [19]:
df = df.loc[~df.Store.isin([291,622,879])]

Continuing with the plan to create new column with month/year this is the code that does that

In [20]:
#part 3 
#year
#we fill the na Year with 2013 
temp_osy = (df.CompetitionOpenSinceYear.fillna(2013)).copy()

temp_osy = temp_osy.astype(str).str.split('.',expand = True)[0]

#month
temp_osm = (df.CompetitionOpenSinceMonth.fillna(1)).copy()
temp_osm = temp_osm.astype(str).str.split('.',expand = True)[0]
#datetime requires month as a zero-padded decimal number.
bad_month = ['1','2','3','4','5','6','7','8','9']
temp_osm = temp_osm.apply(lambda x: '0'+x if  (x in bad_month) else x       )

https://stackoverflow.com/questions/49164935/how-to-use-loc-with-groupby-and-two-conditions-in-pandas

Now we combine to new column then substract to find the months since competion opened 

In [21]:
#part 4 create new column then substract by date column 
#to find months since competition opened for a given date

competition_date = pd.to_datetime(temp_osy + temp_osm,format = '%Y%m')

Overflow

In [22]:
competition_date.describe()

count                 1014567
unique                    172
top       2013-01-01 00:00:00
freq                   320706
first     1900-01-01 00:00:00
last      2015-08-01 00:00:00
Name: 0, dtype: object

In [23]:
competition_months_open = ((df.Date.dt.year - competition_date.dt.year)*12
 + (df.Date.dt.month - competition_date.dt.month) )

In [24]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [25]:
competition_months_open.describe()

count   1014567.00
mean         45.95
std          63.65
min         -31.00
25%          10.00
50%          26.00
75%          73.00
max        1386.00
dtype: float64

In [26]:
competition_months_open[competition_months_open<0] = 0

In [27]:
df["competition_months_open"]= competition_months_open

In [30]:
df.to_csv('rossman_data.csv',index = False)